<a href="https://colab.research.google.com/github/DataScientistMSBA/ML_AI/blob/main/OpenAI_(GPT_3)_Company_Wikipedia_Data_Extractor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Install Packages**

In [1]:
from IPython.display import clear_output
!pip install openai
clear_output(wait=True)
print('All packages installed')

All packages installed


**Import Packages**

In [6]:
import os
import openai
import re
import urllib.request 
from urllib.request import urlopen
from bs4 import BeautifulSoup
import requests
import pandas as pd
from google.colab import drive
import json

clear_output(wait=True)
print('All packages loaded')

All packages loaded


**Lookup and Assign API Key**

In [3]:
drive.mount('/content/gdrive/')

Mounted at /content/gdrive/


In [4]:
cd gdrive/MyDrive/'Colab Notebooks'/Security/

/content/gdrive/MyDrive/Colab Notebooks/Security


In [7]:
fileObject = open("Passwords.txt", "r")
data = fileObject.read()
data = json.loads(data)
openai.api_key = data['OpenAI']['key']
print('API key loaded')

API key loaded


**Compile Company List & Extract Wikipedia URLs using GPT-3**

The code below is a function that takes a list of company names and returns the Wikipedia URL for each company.

In [8]:
CompanyList = ['Facebook', 'Apple', 'Amazon', 'Netflix', 'Google']

Question_WikiURL = [f'''What is the Wikipedia URL for company {i}?''' for i in CompanyList]

n = 0; URL_List = []

while n < len(Question_WikiURL):
  prompt = f'''
  Question: {Question_WikiURL[n]}
  '''
  URL_List.append(prompt)
  n+=1
URL_List = [i.replace('\n','').strip() for i in URL_List]

def WikiURL():
  output = []
  for text in URL_List:
    response = openai.Completion.create(
    model="text-davinci-002",
    prompt=f"{text}?\nA:  ",
    temperature=0,
    max_tokens=100,
    top_p=1,
    frequency_penalty=0,
    presence_penalty=0,
    stop=["\n"])
    output.append(response['choices'][0]['text'])
  return output
output = WikiURL()
output_clean = []
for r in output:
  new = r.strip()
  output_clean.append(new)
df_company = pd.DataFrame(CompanyList)
df_output = pd.DataFrame(output_clean)
df_output = df_company.merge(df_output, how='outer', left_index=True, right_index=True).rename(columns={'0_x':'Company','0_y':'Wikipedia URL'})
df_output

,Company,Wikipedia URL
0,Facebook,https://en.wikipedia.org/wiki/Facebook
1,Apple,https://en.wikipedia.org/wiki/Apple_Inc.
2,Amazon,https://en.wikipedia.org/wiki/Amazon_(company)
3,Netflix,https://en.wikipedia.org/wiki/Netflix
4,Google,https://en.wikipedia.org/wiki/Google


**Compile List of Questions**

In [ ]:
df_questions = []
for company_lookup in df_output['Company']:
  Question1 = f'''How many employees does {company_lookup} have?'''
  Question2 = f'''How many locations does {company_lookup} have?'''
  Question3 = f'''What ranking in the Fortune 500 is {company_lookup}?'''
  Question4 = f'''What is {company_lookup}'s total revenue?'''
  df_questions.append([company_lookup, Question1,Question2, Question3, Question4])
df_questions = pd.DataFrame(df_questions).rename(columns={0:'Company', 1:'Headcount', 2:'Locations', 3:'Fortune500Rank', 4:'Total Revenue'})
df_questions_wURL = df_output.merge(df_questions, how = 'left', on = 'Company')
df_questions_wURL

,Company,Wikipedia URL,Headcount,Locations,Fortune500Rank,Total Revenue
0,Facebook,https://en.wikipedia.org/wiki/Facebook,How many employees does Facebook have?,How many locations does Facebook have?,What ranking in the Fortune 500 is Facebook?,What is Facebook's total revenue?
1,Apple,https://en.wikipedia.org/wiki/Apple_Inc.,How many employees does Apple have?,How many locations does Apple have?,What ranking in the Fortune 500 is Apple?,What is Apple's total revenue?
2,Amazon,https://en.wikipedia.org/wiki/Amazon_(company),How many employees does Amazon have?,How many locations does Amazon have?,What ranking in the Fortune 500 is Amazon?,What is Amazon's total revenue?
3,Netflix,https://en.wikipedia.org/wiki/Netflix,How many employees does Netflix have?,How many locations does Netflix have?,What ranking in the Fortune 500 is Netflix?,What is Netflix's total revenue?
4,Google,https://en.wikipedia.org/wiki/Google,How many employees does Google have?,How many locations does Google have?,What ranking in the Fortune 500 is Google?,What is Google's total revenue?


**Extract All Text from Wikipedia**

In [ ]:
pd.set_option('display.max_colwidth', 100)
n = 0; company_soup = []; PromptList = []

for i in df_questions_wURL['Wikipedia URL']:
  html = urllib.request.urlopen(i)                                                # opening the url for reading
  htmlParse = BeautifulSoup(html, 'html.parser')                                  # parsing the html file
  text = []
  for para in htmlParse.find_all("div"):                                          # getting all the paragraphs
      text.append(para.get_text())
  text = ' '.join(text[2].split('\xa0')).replace('\n','').split('History[edit]')[0]                                                            # Need to find a better solution here
  # text = ' '.join(' '.join(text[2].split('\xa0')).replace('\n','').split()[:4000])
  company_soup.append(text)

df_company_soup = pd.DataFrame(company_soup)

df_questions_wURLnText = df_questions_wURL.merge(df_company_soup, how='outer', left_index=True, right_index=True).rename(columns={0:'Wikipedia Text'})
df_questions_wURLnText

,Company,Wikipedia URL,Headcount,Locations,Fortune500Rank,Total Revenue,Wikipedia Text
0,Facebook,https://en.wikipedia.org/wiki/Facebook,How many employees does Facebook have?,How many locations does Facebook have?,What ranking in the Fortune 500 is Facebook?,What is Facebook's total revenue?,"FacebookFrom Wikipedia, the free encyclopediaJump to navigationJump to searchSocial media servic..."
1,Apple,https://en.wikipedia.org/wiki/Apple_Inc.,How many employees does Apple have?,How many locations does Apple have?,What ranking in the Fortune 500 is Apple?,What is Apple's total revenue?,"Apple Inc.From Wikipedia, the free encyclopediaJump to navigationJump to searchAmerican multinat..."
2,Amazon,https://en.wikipedia.org/wiki/Amazon_(company),How many employees does Amazon have?,How many locations does Amazon have?,What ranking in the Fortune 500 is Amazon?,What is Amazon's total revenue?,"Amazon (company)From Wikipedia, the free encyclopediaJump to navigationJump to searchAmerican mu..."
3,Netflix,https://en.wikipedia.org/wiki/Netflix,How many employees does Netflix have?,How many locations does Netflix have?,What ranking in the Fortune 500 is Netflix?,What is Netflix's total revenue?,"NetflixFrom Wikipedia, the free encyclopediaJump to navigationJump to searchAmerican content pla..."
4,Google,https://en.wikipedia.org/wiki/Google,How many employees does Google have?,How many locations does Google have?,What ranking in the Fortune 500 is Google?,What is Google's total revenue?,"GoogleFrom Wikipedia, the free encyclopediaJump to navigationJump to searchAmerican multinationa..."


In [ ]:
df_questions_wURLnText['Wikipedia Text Trunc'] = [item[:4000] for item in df_questions_wURLnText['Wikipedia Text'].astype(str)]
df_questions_wURLnText

,Company,Wikipedia URL,Headcount,Locations,Fortune500Rank,Total Revenue,Wikipedia Text,Wikipedia Text Trunc
0,Facebook,https://en.wikipedia.org/wiki/Facebook,How many employees does Facebook have?,How many locations does Facebook have?,What ranking in the Fortune 500 is Facebook?,What is Facebook's total revenue?,"FacebookFrom Wikipedia, the free encyclopediaJump to navigationJump to searchSocial media servic...","FacebookFrom Wikipedia, the free encyclopediaJump to navigationJump to searchSocial media servic..."
1,Apple,https://en.wikipedia.org/wiki/Apple_Inc.,How many employees does Apple have?,How many locations does Apple have?,What ranking in the Fortune 500 is Apple?,What is Apple's total revenue?,"Apple Inc.From Wikipedia, the free encyclopediaJump to navigationJump to searchAmerican multinat...","Apple Inc.From Wikipedia, the free encyclopediaJump to navigationJump to searchAmerican multinat..."
2,Amazon,https://en.wikipedia.org/wiki/Amazon_(company),How many employees does Amazon have?,How many locations does Amazon have?,What ranking in the Fortune 500 is Amazon?,What is Amazon's total revenue?,"Amazon (company)From Wikipedia, the free encyclopediaJump to navigationJump to searchAmerican mu...","Amazon (company)From Wikipedia, the free encyclopediaJump to navigationJump to searchAmerican mu..."
3,Netflix,https://en.wikipedia.org/wiki/Netflix,How many employees does Netflix have?,How many locations does Netflix have?,What ranking in the Fortune 500 is Netflix?,What is Netflix's total revenue?,"NetflixFrom Wikipedia, the free encyclopediaJump to navigationJump to searchAmerican content pla...","NetflixFrom Wikipedia, the free encyclopediaJump to navigationJump to searchAmerican content pla..."
4,Google,https://en.wikipedia.org/wiki/Google,How many employees does Google have?,How many locations does Google have?,What ranking in the Fortune 500 is Google?,What is Google's total revenue?,"GoogleFrom Wikipedia, the free encyclopediaJump to navigationJump to searchAmerican multinationa...","GoogleFrom Wikipedia, the free encyclopediaJump to navigationJump to searchAmerican multinationa..."


In [ ]:
df_lookup = df_questions_wURLnText.melt(id_vars=['Company', 'Wikipedia URL', 'Wikipedia Text', 'Wikipedia Text Trunc']).sort_values(by='Company').rename(columns={'variable':'Subject','value':'Question'}).sort_index(axis = 0)

comp=0; complist=[]
while comp < len(df_lookup['Company'].drop_duplicates()):
  df_lookup['Prompt'] = 'Prompt: ' + df_lookup['Wikipedia Text Trunc'].astype(str) + '\n' +'Question: ' + df_lookup['Question'].astype(str)#.split('\n')
  compitem = df_lookup[df_lookup['Company']==df_lookup['Company'].drop_duplicates()[comp]]
  comp+=1
  complist.append(compitem)

def AnalyzeWiki(rec):
  output = []
  for text in complist[rec]['Prompt']:
    response = openai.Completion.create(
    model="text-davinci-002",
    prompt=f"{text}\nA:  ",
    temperature=0,
    max_tokens=100,
    top_p=1,
    frequency_penalty=0,
    presence_penalty=0,
    stop=["\n"])
    output.append(response['choices'][0]['text'])
  rec+=1
  return output

df_output_data = []; rec=0
while rec < df_output['Company'].nunique():
  df_output_data.append(AnalyzeWiki(rec))
  rec+=1
  
df_output_data = pd.DataFrame(df_output_data).rename(columns={0:'Number of Employees',1:'Number of Facilities',2:'Fortune 500 Ranking',3:'Total Revenue'})
df_final_output = df_output.merge(df_output_data, how='outer', left_index=True, right_index=True)
df_final_output

,Company,Wikipedia URL,Number of Employees,Number of Facilities,Fortune 500 Ranking,Total Revenue
0,Facebook,https://en.wikipedia.org/wiki/Facebook,,,Facebook is not currently ranked in the Fortune 500.,
1,Apple,https://en.wikipedia.org/wiki/Apple_Inc.,"154,000",519 retail stores (2022),Apple is currently ranked at number four on the Fortune 500 list.,Apple's total revenue is US$365.82 billion.
2,Amazon,https://en.wikipedia.org/wiki/Amazon_(company),"1,622,000",,Amazon is ranked #3 in the most recent Fortune 500 list.,Amazon's total revenue for 2021 was $469.822 billion.
3,Netflix,https://en.wikipedia.org/wiki/Netflix,"12,135","Netflix is available worldwide aside from Mainland China, Syria, North Korea, and Russia.",Netflix is not currently ranked in the Fortune 500.,US$29.7 billion (2021)
4,Google,https://en.wikipedia.org/wiki/Google,,,4,


**Extract Data From Right Panel**

In [ ]:
def get_Wiki(url):
    # Get Labels:
    r = requests.get(url)
    soup = BeautifulSoup(r.text, 'lxml')
    labels = soup.find('table', {'class': 'infobox vcard'}).find_all('th')
    label_list = []
    for label in labels:
        label_list.append(label.text.strip())
    df = pd.DataFrame(columns=label_list)
    df = list(df)
    df_labels = pd.DataFrame(df).rename(columns={0:'Title'})
    df_labels

    # Get Data:
    page = urlopen(url)
    soup = BeautifulSoup(page, "lxml")
    right_table=soup.find('table', class_='infobox vcard')                      # find the table
    tr_tags = right_table.find_all('tr')                                        # find all the <tr> tag pairs, skip the first one
    data = []                                                                   # create an empty list to hold all the data

    for tr_tag in tr_tags:                                                      # for every row of the table, find the <td> tag pairs and append the text part to the data list
        td_tags = tr_tag.find_all('td')
        row = [tr_tag.text for tr_tag in td_tags]
        data.append(row)

    df = pd.DataFrame(data)                                                     # convert the data into a pandas DataFrame
    df = pd.DataFrame(df[2:-1].reset_index()[0]).rename(columns={0:'Data'})
    df.drop(df.index[(df["Data"].str.contains("List of"))],axis=0,inplace=True)
    df_data = pd.DataFrame(df.reset_index()['Data'])
    
    # Merge Final Output:
    df_final = df_labels.merge(df_data, how = 'outer', left_index=True, right_index=True)
    return df_final

In [ ]:
num = 4
print(df_output['Wikipedia URL'][num])
get_Wiki(df_output['Wikipedia URL'][num])

https://en.wikipedia.org/wiki/Google


,Title,Data
0,Formerly,Google Inc. (1998–2017)
1,Type,Subsidiary (LLC)
2,Industry,Artificial intelligenceAdvertisingCloud comput...
3,Founded,"September 4, 1998; 23 years ago (1998-09-04)[a..."
4,Founders,Larry PageSergey Brin
5,Headquarters,"1600 Amphitheatre Parkway, Mountain View, Cali..."
6,Area served,Worldwide
7,Key people,Sundar Pichai (CEO)Ruth Porat (CFO)Thomas Kuri...
8,Products,"139,995 (2021)"
9,Number of employees,Alphabet Inc.


'https://en.wikipedia.org/wiki/Facebook'